In [1]:
# import library 
import sktime 
from sktime.utils.load_data import load_from_arff_to_dataframe

In [2]:
s#combined dataset 
Xtrain, ytrain = load_from_arff_to_dataframe("../sktime-notebooks/data/Multivariate2018_arff/Multivariate_arff/AtrialFibrillation/AtrialFibrillation_TRAIN.arff")
Xtest, ytest = load_from_arff_to_dataframe("../sktime-notebooks/data/Multivariate2018_arff/Multivariate_arff/AtrialFibrillation/AtrialFibrillation_TEST.arff")

In [3]:
Xtrain.head()

,dim_0,dim_1
0,0 -0.34086 1 -0.38038 2 -0.34580 3...,0 0.14820 1 0.13338 2 0.10868 3...
1,0 -0.11362 1 -0.07410 2 -0.05928 3...,0 -0.00988 1 -0.02470 2 -0.00494 3...
2,0 -0.2079 1 -0.1683 2 -0.1980 3 ...,0 -0.02632 1 -0.04606 2 -0.08554 3...
3,0 -0.11805 1 -0.08657 2 -0.09444 3...,0 0.03510 1 0.04680 2 0.06435 3...
4,0 -0.11362 1 -0.06422 2 -0.05928 3...,0 -0.04940 1 0.01482 2 0.03952 3...


In [4]:
Xtrain.describe()

,dim_0,dim_1
count,15,15
unique,15,15
top,0 0.07686 1 0.06588 2 0.02196 3...,0 0.13090 1 0.13090 2 0.09520 3...
freq,1,1


In [7]:
import numpy as np
np.unique(ytrain)

array(['n', 's', 't'], dtype='<U1')

In [11]:
#first we will run the high level interface 
#to run the high level interface we need the data to be in one table 
import pandas as pd 
ytrain_df= pd.DataFrame(data=ytrain)

In [22]:
ytrain_df=ytrain_df.rename(columns={0 :'target'})

In [23]:
ytrain_df

,target
0,n
1,n
2,n
3,n
4,n
5,s
6,s
7,s
8,s
9,s


In [24]:
# Repeat this for ytest as well
ytest_df= pd.DataFrame(data=ytrain)
ytest_df=ytrain_df.rename(columns={0 :'target'})

In [25]:
# then concat it with the rest of the dataframe
train = pd.concat([Xtrain, ytrain_df],axis=1)
test = pd.concat([Xtest, ytest_df],axis=1)

In [27]:
train

,dim_0,dim_1,target
0,0 -0.34086 1 -0.38038 2 -0.34580 3...,0 0.14820 1 0.13338 2 0.10868 3...,n
1,0 -0.11362 1 -0.07410 2 -0.05928 3...,0 -0.00988 1 -0.02470 2 -0.00494 3...,n
2,0 -0.2079 1 -0.1683 2 -0.1980 3 ...,0 -0.02632 1 -0.04606 2 -0.08554 3...,n
3,0 -0.11805 1 -0.08657 2 -0.09444 3...,0 0.03510 1 0.04680 2 0.06435 3...,n
4,0 -0.11362 1 -0.06422 2 -0.05928 3...,0 -0.04940 1 0.01482 2 0.03952 3...,n
5,0 -0.02964 1 0.00988 2 0.00988 3...,0 0.00000 1 -0.00988 2 -0.02964 3...,s
6,0 0.66445 1 1.16450 2 1.00010 3...,0 0.02684 1 0.08723 2 -0.16775 3...,s
7,0 0.07686 1 0.06588 2 0.02196 3...,0 0.13090 1 0.13090 2 0.09520 3...,s
8,0 -0.2091 1 -0.2091 2 -0.2040 3 ...,0 0.01104 1 -0.00552 2 0.00000 3...,s
9,0 -0.01482 1 -0.03458 2 -0.03458 3...,0 0.01482 1 0.01482 2 0.00494 3...,s


In [29]:
from sktime.highlevel.tasks import TSCTask
from sktime.highlevel.strategies import TSCStrategy
from sktime.classifiers.compose import TimeSeriesForestClassifier

#A task, which encapsulates the information about the learning task, for example the name of the target variable, and any additional necessary instructions on how to run fit and predict.
task = TSCTask(target='target', metadata=train)
clf_high_level = TimeSeriesForestClassifier(n_estimators=50)
#A strategy which wraps the low level estimators and takes a task and the whole dataframe as input in fit.
strategy = TSCStrategy(clf_high_level)

In [30]:
strategy.fit(task, train)

y_pred = strategy.predict(test)
y_test = test[task.target]
accuracy_score(y_test, y_pred)

NotImplementedError: X must be univariate with X.shape[1] == 1, but found: X.shape[1] == 2. For multivariate please use compositor classes. Estimator-specific multivariate approaches are not implemented yet.

In [34]:
#oops that high level interface isn't implemented 
# so we go back to the beginning of all this 
from sktime.transformers.compose import ColumnConcatenator
from sktime.pipeline import Pipeline

steps_concat = [
    ('concatenate', ColumnConcatenator()),
    ('classify', TimeSeriesForestClassifier(n_estimators=100))]
clf_concat = Pipeline(steps_concat)
clf_concat.fit(Xtrain, ytrain)
clf_concat.score(Xtest, ytest)

0.3333333333333333

In [46]:
from sktime.classifiers.dictionary_based.boss import BOSSEnsemble
from sktime.classifiers.compose import ColumnEnsembleClassifier
clf_ensemble = ColumnEnsembleClassifier(estimators=[
    ("TSF0", TimeSeriesForestClassifier(n_estimators=100), [0]),
    ("BOSSEnsemble2", BOSSEnsemble(ensemble_size=2), [2]),
])
clf_ensemble.fit(Xtrain, ytrain)
clf_ensemble.score(Xtest, ytest)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [47]:
from sktime.classifiers.shapelet_based import ShapeletTransformClassifier

clf_shapelet = ShapeletTransformClassifier(time_contract_in_mins=0.7)
clf_shapelet.fit(Xtrain, ytrain)
clf_shapelet.score(Xtest, ytest)

0.2

In [41]:
Xtrain.head()

,dim_0,dim_1
0,0 -0.34086 1 -0.38038 2 -0.34580 3...,0 0.14820 1 0.13338 2 0.10868 3...
1,0 -0.11362 1 -0.07410 2 -0.05928 3...,0 -0.00988 1 -0.02470 2 -0.00494 3...
2,0 -0.2079 1 -0.1683 2 -0.1980 3 ...,0 -0.02632 1 -0.04606 2 -0.08554 3...
3,0 -0.11805 1 -0.08657 2 -0.09444 3...,0 0.03510 1 0.04680 2 0.06435 3...
4,0 -0.11362 1 -0.06422 2 -0.05928 3...,0 -0.04940 1 0.01482 2 0.03952 3...


In [43]:
ytrain

array(['n', 'n', 'n', 'n', 'n', 's', 's', 's', 's', 's', 't', 't', 't',
       't', 't'], dtype='<U1')

In [1]:
clf = ColumnEnsembleClassifier(estimators=[
    ("TSF0", TimeSeriesForestClassifier(n_estimators=100), [0]),
    ("BOSSEnsemble3", BOSSEnsemble(ensemble_size=3), [3]),
])
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

NameError: name 'ColumnEnsembleClassifier' is not defined